Data Requirement:
1. Both metagenomics and metabolomics data is in SEPARATE csv files as xxx.csv
2. The samples are columns and the microbe/metabolite are rows in both files
3. The first row (header is the name of samples)
4. The first column are names of microbe/metabolite

Metabolomics: 
1. The csv file is preprocessed
2. The data is logged
3. There is no missingness value - data is imputed using some method 

Metagenomics
1. The data can be either count or relative abundance
2. The data should NOT have any kind of transformation (clr, box-cox, etc)

To do: Please correct the data link before running (Box 3/Code block 2)

In [1]:
import pandas as pd
import numpy as np
import os
import shutil
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages

import M2module
import M2dataproccess

from sklearn.exceptions import ConvergenceWarning
import warnings
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=ConvergenceWarning)

In [2]:
# !Please load data paths here 
microbiome_path = 'biocrust/data/microbiome.csv'
metabolome_path = 'biocrust/data/metabolome.csv'

In [3]:
# Create the folder if it doesn't exist
folder_name = 'twinsuk'
if not os.path.exists(folder_name):
    os.makedirs(folder_name)
else:
    shutil.rmtree(folder_name)
    os.makedirs(folder_name)

#create data folder for storing result file in twinsuk folder
results_folder = os.makedirs('twinsuk/results')

In [4]:
# Load metabolome
met, met_header = M2dataproccess.parse_raw_data(metabolome_path)

# Transpose the metabolome data
met_t = M2dataproccess.transpose_csv(met)

# Convert to numeric and fill NaN with zeros
metabolome_data = met_t.apply(pd.to_numeric, errors='coerce').fillna(0)

# Get all metabolites
all_metabolites = metabolome_data.columns

# Save plots to a PDF with multiple plots per page
with PdfPages('twinsuk/metabolite_distributions_multi.pdf') as pdf:
    figs_per_page = 8  # Number of plots per page (2 rows x 4 columns)
    rows, cols = 2, 4
    total_plots = len(all_metabolites)
    
    for i in range(0, total_plots, figs_per_page):
        plt.figure(figsize=(16, 10))  # Size for each page
        for j in range(figs_per_page):
            idx = i + j
            if idx >= total_plots:  # Avoid overflow
                break
            metabolite = all_metabolites[idx]
            plt.subplot(rows, cols, j + 1)
            sns.histplot(metabolome_data[metabolite], kde=True, bins=30)
            plt.title(f'Distribution of {metabolite}')
            plt.xlabel('Abundance')
            plt.ylabel('Frequency')

        plt.tight_layout()
        pdf.savefig()  # Save current page to PDF
        plt.close()


In [7]:

# Define thresholds to loop through
threshold_values = [0.1, 0.2, 0.25, 0.3]

# Loop through thresholds
for threshold in threshold_values:
    print(f"Processing for threshold: {threshold}")

    # Load and process microbiome data
    mic, mic_header = M2dataproccess.parse_raw_data(microbiome_path)

    # Transpose the data and drop rare features based on threshold
    mic_t = M2dataproccess.transpose_csv(mic)
    microbiome = M2dataproccess.drop_rare_features(mic_t, threshold=threshold)

    # Convert microbiome data to numeric and fill NaN with zeros
    microbiome_data = microbiome.apply(pd.to_numeric, errors='coerce').fillna(0)

    # Apply CLR transformation and power transformation
    raw = M2dataproccess.make_compositional(microbiome_data, 'none', 'none')
    microbiome_clr = M2dataproccess.make_compositional(microbiome_data, 'clr', 'none')
    microbiome_pt = M2dataproccess.make_compositional(microbiome_data, 'none', 'power')
    microbiome_pt_clr = M2dataproccess.make_compositional(microbiome_data, 'clr', 'power')

    # Dictionary to hold the microbiome versions
    microbiome_versions = {
        'raw': raw,
        'clr': microbiome_clr,
        'pt': microbiome_pt,
        'pt_clr': microbiome_pt_clr
    }

    # Save plots for each microbiome version
    all_species = microbiome_data.columns
    pdf_path = f'twinsuk/microbe_distributions_threshold_{threshold}.pdf'
    
    with PdfPages(pdf_path) as pdf:
        microbes_per_page = 2  # Two microbes per page
        variations = ['Raw', 'CLR', 'Power', 'CLR+Power']
        rows, cols = 2, 4  # Each microbe has 4 columns for its variations
        
        total_plots = len(all_species)
        
        for i in range(0, total_plots, microbes_per_page):
            plt.figure(figsize=(20, 10))  # Size for each page
            for j in range(microbes_per_page):
                idx = i + j
                if idx >= total_plots:  # Avoid overflow
                    break
                species = all_species[idx]
                
                # Create subplots for the current microbe's variations
                for k, data in enumerate([microbiome_data, microbiome_clr, microbiome_pt, microbiome_pt_clr]):
                    plt.subplot(rows, cols, j * 4 + k + 1)  # Offset for the microbe (row position)
                    sns.histplot(data[species], kde=True, bins=30)
                    plt.title(f'{variations[k]} - {species}')
                    plt.xlabel('Value')
                    plt.ylabel('Frequency')

            plt.tight_layout()
            pdf.savefig()  # Save current page to PDF
            plt.close()
    print(f"Plots saved to {pdf_path}")
    
    os.makedirs('twinsuk/results/threshold_{}'.format(threshold))

    # Loop through each microbiome version and align with metabolite data
    for version_name, microbiome in microbiome_versions.items():
        
        # Align microbiome and metabolite data
        microbe, metabolite = M2dataproccess.align_microbiome_metabolite(microbiome, metabolome_data)
        print(f"Aligned microbiome shape: {microbe.shape}")
        print(f"Aligned metabolome shape: {metabolite.shape}")

        # Perform the MB algorithm and regress microbiome on metabolome and vice versa
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore", category=ConvergenceWarning) 
            
            # Perform neighborhood selection
            microbiome_neighborhoods = M2module.mb_neighborhood_selection(microbe, metabolite)
            metabolites_neighborhoods = M2module.mb_neighborhood_selection(metabolite, microbe)

        # Renaming the rows and columns for clarity
        microbiome_neighborhoods.index = microbe.columns 
        microbiome_neighborhoods.columns = metabolite.columns

        metabolites_neighborhoods.index = metabolite.columns
        metabolites_neighborhoods.columns = microbe.columns
        
        # Save the results to CSV files for each microbiome version and threshold
        microbiome_csv_path = f'twinsuk/results/threshold_{threshold}/{version_name}_microbe_neighborhood.csv'
        metabolite_csv_path = f'twinsuk/results/threshold_{threshold}/{version_name}__metabolite_neighborhood.csv'
        
        microbiome_neighborhoods.to_csv(microbiome_csv_path)
        metabolites_neighborhoods.to_csv(metabolite_csv_path)
        
        print(f"Saved microbiome neighborhood to {microbiome_csv_path}")
        print(f"Saved metabolite neighborhood to {metabolite_csv_path}")


Processing for threshold: 0.1
Number of microbes before dropping: 466
Number of microbes after dropping: 455
Plots saved to twinsuk/microbe_distributions_threshold_0.1.pdf
Aligned microbiome shape: (19, 455)
Aligned metabolome shape: (19, 85)
Saved microbiome neighborhood to twinsuk/results/threshold_0.1/raw_microbe_neighborhood.csv
Saved metabolite neighborhood to twinsuk/results/threshold_0.1/raw__metabolite_neighborhood.csv
Aligned microbiome shape: (19, 455)
Aligned metabolome shape: (19, 85)
Saved microbiome neighborhood to twinsuk/results/threshold_0.1/clr_microbe_neighborhood.csv
Saved metabolite neighborhood to twinsuk/results/threshold_0.1/clr__metabolite_neighborhood.csv
Aligned microbiome shape: (19, 455)
Aligned metabolome shape: (19, 85)
Saved microbiome neighborhood to twinsuk/results/threshold_0.1/pt_microbe_neighborhood.csv
Saved metabolite neighborhood to twinsuk/results/threshold_0.1/pt__metabolite_neighborhood.csv
Aligned microbiome shape: (19, 455)
Aligned metabolom